In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

In [ ]:
import json
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Define the file paths for the secure connect bundle and token JSON file
SECURE_CONNECT_BUNDLE = 'secure-connect-bd-db.zip'
TOKEN_FILE = 'BD_DB-token.json'

# Load secrets from the token JSON file
with open(TOKEN_FILE, 'r') as token_file:
    secrets = json.load(token_file)

# Extract client credentials
CLIENT_ID = secrets.get("clientId")
CLIENT_SECRET = secrets.get("secret")

# Configure authentication provider
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)

# Set up cloud configuration and create a cluster connection
cloud_config = {'secure_connect_bundle': SECURE_CONNECT_BUNDLE}
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)

# Connect to the session
session = cluster.connect()

# Check connection status
if session:
    print('Connected!')
else:
    print('An error occurred while connecting.')



Connected!


In [ ]:
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 31.0 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd

# Define the URL for the CSV file and the local filename to save it
CSV_URL = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
LOCAL_FILENAME = "sales_100.csv"

try:
    # Send an HTTP GET request to download the CSV file
    response = requests.get(CSV_URL)
    response.raise_for_status()  # Ensure the request was successful

    # Write the downloaded data to a local file
    with open(LOCAL_FILENAME, 'wb') as file:
        file.write(response.content)

    print(f"File successfully downloaded and saved as '{LOCAL_FILENAME}'.")

    # Read the CSV file into a pandas DataFrame
    sales_data = pd.read_csv(LOCAL_FILENAME)

    # Display the first few rows of the DataFrame
    print("Preview of the first 5 rows in the dataset:")
    print(sales_data.head())

except requests.RequestException as e:
    print(f"Error occurred while downloading the file: {e}")
except pd.errors.EmptyDataError:
    print("The downloaded CSV file is empty.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



File successfully downloaded and saved as 'sales_100.csv'.
Preview of the first 5 rows in the dataset:
                         Region           Country  Item Type Sales Channel  \
0            Sub-Saharan Africa      South Africa     Fruits       Offline   
1  Middle East and North Africa           Morocco    Clothes        Online   
2         Australia and Oceania  Papua New Guinea       Meat       Offline   
3            Sub-Saharan Africa          Djibouti    Clothes       Offline   
4                        Europe          Slovakia  Beverages       Offline   

  Order Priority  Order Date   Order ID   Ship Date  UnitsSold  UnitPrice  \
0              M   7/27/2012  443368995   7/28/2012       1593       9.33   
1              M   9/14/2013  667593514  10/19/2013       4611     109.28   
2              M   5/15/2015  940995585    6/4/2015        360     421.89   
3              H   5/17/2017  880811536    7/2/2017        562     109.28   
4              L  10/26/2016  174590194   1

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS cassandra.bronze_sales (
    id UUID PRIMARY KEY,
    transaction_id TEXT,
    customer_id TEXT,
    product_id TEXT,
    amount DECIMAL,
    transaction_date TEXT
);
""")


In [ ]:
from uuid import uuid4

In [15]:
sales_data.columns = sales_data.columns.str.strip().str.replace(' ', '_')


In [17]:
from datetime import datetime
from uuid import uuid4

# Loop through each row in the DataFrame
for _, row in sales_data.iterrows(): # Changed 'dataframe' to 'sales_data'
    try:
        # Convert the 'Order_Date' column value to a datetime object and reformat it
        formatted_date = datetime.strptime(row['Order_Date'], '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        # Handle invalid date formats by skipping the current row
        print(f"Encountered an invalid date format: {row['Order_Date']}")
        continue

    # Insert data into the Cassandra table
    session.execute("""
    INSERT INTO cassandra.bronze_sales (
        id, transaction_id, customer_id, product_id, amount, transaction_date
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        uuid4(),                              # Generate a unique identifier for the primary key
        str(row['Order_ID']),                 # Convert Order_ID to a string
        str(row['Region']),                   # Convert Region to a string
        str(row['Item_Type']),                # Convert Item_Type to a string
        float(row['TotalRevenue']),           # Ensure TotalRevenue is stored as a float
        formatted_date                        # Use the reformatted date string
    ))



In [18]:
from datetime import datetime

# Create Silver Table
session.execute("""
CREATE TABLE IF NOT EXISTS cassandra.silver_sales (
    transaction_id text PRIMARY KEY,
    customer_id text,
    product_id text,
    amount decimal,
    transaction_date date
)
""")

# Transform Data for Silver Table
query = "SELECT transaction_id, customer_id, product_id, amount, transaction_date FROM cassandra.bronze_sales"
rows = session.execute(query)

# Insert data into silver_sales
for row in rows:
    # Convert 'transaction_date' to 'YYYY-MM-DD' format (only the date part)
    try:
        # If the date is in 'YYYY-MM-DD HH:MM:SS' format, extract only the date part
        transaction_date = datetime.strptime(row.transaction_date, '%Y-%m-%d %H:%M:%S').date()
    except ValueError:
        # If the 'transaction_date' is already in 'YYYY-MM-DD' format, just parse it
        transaction_date = datetime.strptime(row.transaction_date, '%Y-%m-%d').date()

    # Insert the data into the silver_sales table
    session.execute("""
    INSERT INTO cassandra.silver_sales (transaction_id, customer_id, product_id, amount, transaction_date)
    VALUES (%s, %s, %s, %s, %s)
    """, (
        row.transaction_id,
        row.customer_id,
        row.product_id,
        row.amount,
        transaction_date
    ))


In [19]:
from collections import defaultdict
from decimal import Decimal

In [21]:
from collections import defaultdict

# Create Table 1: Total Sales by Customer
session.execute("""
CREATE TABLE IF NOT EXISTS cassandra.gold_sales_by_customer (
    customer_id TEXT PRIMARY KEY,
    total_sales DECIMAL
);
""")

# Retrieve data from the silver_sales table to calculate sales by customer
query = "SELECT customer_id, amount FROM cassandra.silver_sales"
rows = session.execute(query)

# Use a defaultdict to sum up sales for each customer
sales_by_customer = defaultdict(float)
for row in rows:
    sales_by_customer[row.customer_id] += float(row.amount)  # Ensure amount is a float

# Insert aggregated data into gold_sales_by_customer
for customer_id, total_sales in sales_by_customer.items():
    session.execute("""
    INSERT INTO cassandra.gold_sales_by_customer (customer_id, total_sales)
    VALUES (%s, %s);
    """, (customer_id, total_sales))

# Create Table 2: Total Sales by Product
session.execute("""
CREATE TABLE IF NOT EXISTS cassandra.gold_sales_by_product (
    product_id TEXT PRIMARY KEY,
    total_sales DECIMAL
);
""")

# Retrieve data from the silver_sales table to calculate sales by product
query = "SELECT product_id, amount FROM cassandra.silver_sales"
rows = session.execute(query)

# Use a defaultdict to sum up sales for each product
sales_by_product = defaultdict(float)
for row in rows:
    sales_by_product[row.product_id] += float(row.amount)  # Ensure amount is a float

# Insert aggregated data into gold_sales_by_product
for product_id, total_sales in sales_by_product.items():
    session.execute("""
    INSERT INTO cassandra.gold_sales_by_product (product_id, total_sales)
    VALUES (%s, %s);
    """, (product_id, total_sales))

# Create Table 3: Daily Sales Summary
session.execute("""
CREATE TABLE IF NOT EXISTS cassandra.gold_daily_sales (
    transaction_date DATE PRIMARY KEY,
    total_sales DECIMAL
);
""")

# Retrieve data from the silver_sales table to calculate daily sales
query = "SELECT transaction_date, amount FROM cassandra.silver_sales"
rows = session.execute(query)

# Use a defaultdict to sum up sales by transaction_date
sales_by_date = defaultdict(float)
for row in rows:
    sales_by_date[row.transaction_date] += float(row.amount)  # Ensure amount is a float

# Insert aggregated data into gold_daily_sales
for transaction_date, total_sales in sales_by_date.items():
    session.execute("""
    INSERT INTO cassandra.gold_daily_sales (transaction_date, total_sales)
    VALUES (%s, %s);
    """, (transaction_date, total_sales))


In [22]:
# Golden Table 1: Total Sales by Customer
result_customer = session.execute("SELECT * FROM cassandra.gold_sales_by_customer")

# Iterate through and print each row
row_iterator = iter(result_customer)  # Create an iterator from the result
while True:
    try:
        current_row = next(row_iterator)  # Retrieve the next row
        print(current_row)  # Display the row
    except StopIteration:
        break  # Stop the loop when there are no more rows to process



Row(customer_id='Australia and Oceania', total_sales=Decimal('10711258.13'))
Row(customer_id='Europe', total_sales=Decimal('34964749.830000006'))
Row(customer_id='Middle East and North Africa', total_sales=Decimal('24765127.25'))
Row(customer_id='Central America and the Caribbean', total_sales=Decimal('17570835.42'))
Row(customer_id='Asia', total_sales=Decimal('28840812.190000005'))
Row(customer_id='Sub-Saharan Africa', total_sales=Decimal('24225437.419999998'))
Row(customer_id='North America', total_sales=Decimal('3611757.5199999996'))


In [23]:
# Golden Table 2: Total Sales by Product
result_product = session.execute("SELECT * FROM cassandra.gold_sales_by_product")

# Iterate through the result and display each row
product_iterator = iter(result_product)  # Create an iterator from the query result
while True:
    try:
        current_product = next(product_iterator)  # Fetch the next row
        print(current_product)  # Output the row
    except StopIteration:
        break  # Exit the loop when all rows have been processed


Row(product_id='Household', total_sales=Decimal('38519082.8'))
Row(product_id='Office Supplies', total_sales=Decimal('27880904.94'))
Row(product_id='Vegetables', total_sales=Decimal('1135114.08'))
Row(product_id='Snacks', total_sales=Decimal('2193642.66'))
Row(product_id='Personal Care', total_sales=Decimal('3191147.8499999996'))
Row(product_id='Meat', total_sales=Decimal('21278865.93'))
Row(product_id='Fruits', total_sales=Decimal('615033.6'))
Row(product_id='Beverages', total_sales=Decimal('2145024.7'))
Row(product_id='Cereal', total_sales=Decimal('9416123.2'))
Row(product_id='Cosmetics', total_sales=Decimal('28727100.399999995'))
Row(product_id='Baby Food', total_sales=Decimal('5200564.159999999'))
Row(product_id='Clothes', total_sales=Decimal('4387373.4399999995'))


In [24]:
# Golden Table 3: Daily Sales Summary
result_daily_sales = session.execute("SELECT * FROM cassandra.gold_daily_sales")

# Loop through and display each row in the result
daily_sales_iterator = iter(result_daily_sales)  # Create an iterator for the query result
while True:
    try:
        current_row = next(daily_sales_iterator)  # Fetch the next row
        print(current_row)  # Print the row
    except StopIteration:
        break  # End the loop when all rows have been processed


Row(transaction_date=Date(16267), total_sales=Decimal('5608790.11'))
Row(transaction_date=Date(15042), total_sales=Decimal('167640.85'))
Row(transaction_date=Date(15162), total_sales=Decimal('289426.4'))
Row(transaction_date=Date(17303), total_sales=Decimal('61415.36'))
Row(transaction_date=Date(16781), total_sales=Decimal('33410.73'))
Row(transaction_date=Date(16769), total_sales=Decimal('643018.2'))
Row(transaction_date=Date(14710), total_sales=Decimal('1932962.9'))
Row(transaction_date=Date(15293), total_sales=Decimal('1456356.0'))
Row(transaction_date=Date(15962), total_sales=Decimal('503890.08'))
Row(transaction_date=Date(16151), total_sales=Decimal('4003440.4'))
Row(transaction_date=Date(16110), total_sales=Decimal('74957.22'))
Row(transaction_date=Date(15218), total_sales=Decimal('6666661.52'))
Row(transaction_date=Date(15389), total_sales=Decimal('217368.45'))
Row(transaction_date=Date(17135), total_sales=Decimal('70036.2'))
Row(transaction_date=Date(17048), total_sales=Decimal